In [2]:
import pandas as pd
from autotind.db import PersonRepo

In [3]:
repo = PersonRepo('sqlite:///tind.sqlite')

persons = repo.get_all()

In [4]:
df = pd.DataFrame(persons)

In [5]:
df['label'] = df['label'].replace('match', 'like')

In [10]:
df = df[df['label'] != 'recommendation']

In [39]:
sample_size = min(len(df[df['label'] == 'like']), len(df[df['label'] == 'dislike']))

groups = df.groupby('label').apply(lambda x: x.sample(sample_size)).reset_index(drop=True)

In [40]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(groups, test_size=0.2)


In [7]:
import torch
torch.cuda.is_available()

False